In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from utils import load_data
from constants import *
import pandas as pd

In [7]:
YEARS = [2014, 2015]
reddit = {}

In [18]:
for year in YEARS:
    reddit[year] = load_data(DATA_PATH, year=year, tokenize=False, comp="bz2", dev=True)

Loading data of 2014...


TypeError: data type 'string[pyarrow]' not understood

In [ ]:
users = {}


for year in YEARS:
    for row in reddit[year]:
        if not row['subreddit'] in DEM_SUBREDDITS and not row['subreddit'] in REP_SUBREDDITS:
            continue
        else:
            try:
                users[row['author']].append(row['subreddit'])
            except KeyError:
                users[row['author']] = [row['subreddit']]
        

In [ ]:
def rep_or_dem(subreddits_list):
    score = 0
    for subreddit in subreddits_list:
        if subreddit in DEM_SUBREDDITS:
            score += 1
        elif subreddit in REP_SUBREDDITS:
            score -= 1
    if score > 0: 
        return 'D'
    elif score < 0:
        return 'R'
    return 'N'

In [ ]:
data = []
for year in YEARS:
    print('YEAR -> 'year)
    for row in reddit[year]:
        if row['subreddit'] == 'politics':
            try:
                affiliation = rep_or_dem(users[row['author']])
                new_line = [row['author'], row['body'], row['created_utc'], affiliation]
                data.append(new_line)
            except KeyError:
                continue
                
output_dataframe = pd.DataFrame(data, columns= ['author', 'post', 'time', 'politics'])
output_dataframe.to_csv('reduced_data.csv')